In [1]:
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
from matplotlib.collections import LineCollection
import cmocean as cmo

In [11]:
moviename = 'BMB_ufelad_full_GL_2km'
tfinished = 36 # 0

In [3]:
vmax = 100
vmin = -10
linthresh = .3
linscale = .25

fracpos = (np.log10(vmax/linthresh)+linscale)/(np.log10(vmax/linthresh)+np.log10(-(vmin/linthresh))+2*linscale)
nneg = np.int_((1-fracpos)*256)

colors1 = plt.get_cmap('cmo.dense_r')(np.linspace(0,1.,nneg+1))
colors2 = plt.get_cmap('gist_heat_r')(np.linspace(0., 1, 256-nneg-1))

# combine them and build a new colormap
colors = np.vstack((colors1, colors2))
mymap = mpl.colors.LinearSegmentedColormap.from_list('my_colormap', colors)

norm=mpl.colors.SymLogNorm(linthresh, vmin=vmin, vmax=vmax, linscale=linscale)

In [12]:
Ncol = 10

#Get all line segments
ds = xr.open_dataset('../output/MISMIPplus_5km_runtime/main_output_ANT_00003.nc')

allsegments = np.zeros((len(ds.vi),len(ds.ci),Ncol-1,2,2))
for v in ds.vi:
    for n in range(ds.nC[v].data):
        x = np.linspace(ds.V[0,v],ds.V[0,ds.C[n,v]-1],Ncol)
        y = np.linspace(ds.V[1,v],ds.V[1,ds.C[n,v]-1],Ncol)
        
        points = np.array([x, y]).T.reshape(-1, 1, 2)
        allsegments[v,n,:,:,:] = np.concatenate([points[:-1], points[1:]], axis=1)


In [13]:
#Make frames
for t in range(len(ds.time)):
    fig,ax = plt.subplots(1,1,figsize=(10,5))

    for v in ds.vi:
        for n in range(ds.nC[v].data):
            cols = np.linspace(-ds.BMB[t,v],-ds.BMB[t,ds.C[n,v]-1],Ncol)
            lc = LineCollection(allsegments[v,n,:,:,:], cmap=mymap,norm=norm)
            lc.set_array(cols)
            lc.set_linewidth(.2)
            line = ax.add_collection(lc)

    ax.set_xlim(4e5,6.5e5)
    #ax.set_xlim(0e5,8.5e5)
    ax.set_ylim(-4e4,4e4)
    ax.set_aspect(1)
    ax.set_title(f"Year {ds.time[t].data-ds.time[0].data:10.2f}")

    ticks = [-10,-3,-1,-.3,0,.3,1,3,10,30,100]
    cb = fig.colorbar(line,ax=ax,extend='both',shrink=.5)
    cb.set_ticks(ticks)
    cb.set_ticklabels(ticks)

    plt.tight_layout()
    plt.savefig(f"../video/frame_{tfinished:03d}")
    plt.close()
    print(f"Finished frame {tfinished:03d}")
    tfinished += 1


Finished frame 036
Finished frame 037
Finished frame 038
Finished frame 039
Finished frame 040


In [14]:
#Make video
os.system(f'ffmpeg -r 4 -f image2 -i ../video/frame_%03d.png -pix_fmt yuv420p ../video/{moviename}.mp4')
#os.system(f'rm -r ../video/frame*.png')

ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.1.1_2 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopenvin

0